In [ ]:
!pip -qq install scikit-learn==0.21.2
!pip -qq install optuna

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
import lightgbm as lgb
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import keras
from sklearn import metrics
import optuna

df = pd.read_csv('/content/Churn.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
df.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

In [ ]:
categorical_cols = ['gender', 
          'SeniorCitizen',
          'Partner', 
          'Dependents', 
          'PhoneService', 
          'MultipleLines', 
          'InternetService', 
          'OnlineSecurity', 
          'OnlineBackup', 
          'DeviceProtection', 
          'TechSupport', 
          'StreamingTV', 
          'StreamingMovies', 
          'Contract', 
          'PaperlessBilling', 
          'PaymentMethod', 
          'Churn'  ]

In [ ]:
le = LabelEncoder()

# apply le on categorical feature columns
df[categorical_cols] = df[categorical_cols].apply(lambda col: le.fit_transform(col))
df[categorical_cols].head(10)

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(np.mean(df['TotalCharges']))

In [ ]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [ ]:
df.iloc[:, -1].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [ ]:
test = df.iloc[4000:]
train = df.iloc[:4000]

print(train.shape)
print(test.shape)

(4000, 21)
(3043, 21)


In [ ]:
train.iloc[:, -1].value_counts()

0    2943
1    1057
Name: Churn, dtype: int64

In [ ]:
y_train = train.iloc[:, -1]
X_train = train.iloc[:, :-1]

y_test = test.iloc[:, -1]
X_test = test.iloc[:, :-1]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4000, 20) (4000,)
(3043, 20) (3043,)


In [ ]:
X, y = SMOTE().fit_resample(X_train.iloc[:, 1:], y_train)

print(X.shape, y.shape)

(5886, 19) (5886,)


In [ ]:
'''
def objective(trial):    
    params = {
                'learning_rate': trial.suggest_uniform('learning_rate', 0.00001, 1.0),
                'max_depth': trial.suggest_int('max_depth', 1, 50),
                #'max_delta_step': trial.suggest_int('max_delta_step', 1, 10000),
                # trial.suggest_int('n_estimators', 1, 10000),
                'random_state': trial.suggest_int('random_state', 1, 10000),
                #'num_parallel_tree': trial.suggest_int('num_parallel_tree', 1, 1000),
                'gamma': trial.suggest_uniform('gamma', 0.00001, 1.0),
                'min_child_weight': trial.suggest_uniform('min_child_weight', 0.00001, 1.0),
                'subsample': trial.suggest_uniform('subsample', 0.00001, 1.0),
                'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.00001, 1.0),
                'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.00001, 1.0),
                'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.00001, 1.0),
                'reg_alpha': trial.suggest_uniform('reg_alpha', 0.00001, 1.0),
                'reg_lambda': trial.suggest_uniform('reg_lambda', 0.00001, 1.0),
                #'min_child_weight': trial.suggest_int('min_child_weight', 1, 10000),
                'max_leaves': trial.suggest_int('max_leaves', 1, 10000),
              
              
                "objective": "binary:logistic",
                'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
            }


    model = xgb.XGBClassifier(**params)
    model.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=100)
    val_pred = model.predict(X_test.iloc[:, 1:].values)
    score = metrics.roc_auc_score(y_test, val_pred)
    print(f"rmse = {score}")
    return score

import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))
'''

[I 2021-02-23 10:37:58,756] A new study created in memory with name: no-name-dde2dd16-4bfd-44e0-a63f-06a789ede7d5


[0]	validation_0-auc:0.62696
Will train until validation_0-auc hasn't improved in 1000 rounds.
[100]	validation_0-auc:0.829169
[200]	validation_0-auc:0.837183
[300]	validation_0-auc:0.838463
[400]	validation_0-auc:0.841964
[500]	validation_0-auc:0.839855
[600]	validation_0-auc:0.84176
[700]	validation_0-auc:0.840674
[800]	validation_0-auc:0.837801
[900]	validation_0-auc:0.836472
[999]	validation_0-auc:0.840403


[I 2021-02-23 10:38:00,531] Trial 0 finished with value: 0.7403144892943808 and parameters: {'learning_rate': 0.26758972998283104, 'max_depth': 1, 'random_state': 9505, 'gamma': 0.8313409099038996, 'min_child_weight': 0.33106549591340845, 'subsample': 0.11742112944496136, 'colsample_bytree': 0.903490440643311, 'colsample_bylevel': 0.5828214273176718, 'colsample_bynode': 0.22834868493918506, 'reg_alpha': 0.4622538514064514, 'reg_lambda': 0.6643312970844757, 'max_leaves': 7545}. Best is trial 0 with value: 0.7403144892943808.


rmse = 0.7403144892943808
[0]	validation_0-auc:0.790149
Will train until validation_0-auc hasn't improved in 1000 rounds.
[100]	validation_0-auc:0.833403
[200]	validation_0-auc:0.826633
[300]	validation_0-auc:0.822619
[400]	validation_0-auc:0.820163
[500]	validation_0-auc:0.816519
[600]	validation_0-auc:0.81249
[700]	validation_0-auc:0.805689
[800]	validation_0-auc:0.803479
[900]	validation_0-auc:0.800033
[999]	validation_0-auc:0.798033


[I 2021-02-23 10:38:05,345] Trial 1 finished with value: 0.7425989692929676 and parameters: {'learning_rate': 0.07077481257550318, 'max_depth': 7, 'random_state': 2390, 'gamma': 0.19956497199344142, 'min_child_weight': 0.24567801653183885, 'subsample': 0.07610165760413587, 'colsample_bytree': 0.7970456781907065, 'colsample_bylevel': 0.8433470774361337, 'colsample_bynode': 0.44113817406720374, 'reg_alpha': 0.22584291413649163, 'reg_lambda': 0.9176794977632119, 'max_leaves': 4944}. Best is trial 1 with value: 0.7425989692929676.


...

rmse = 0.7138278246738192
[0]	validation_0-auc:0.683822
Will train until validation_0-auc hasn't improved in 1000 rounds.
[100]	validation_0-auc:0.826783
[200]	validation_0-auc:0.833943
[300]	validation_0-auc:0.835336
[400]	validation_0-auc:0.836452
[500]	validation_0-auc:0.836237
[600]	validation_0-auc:0.837208
[700]	validation_0-auc:0.836964
[800]	validation_0-auc:0.836982
[900]	validation_0-auc:0.837137
[999]	validation_0-auc:0.837644


[I 2021-02-23 10:45:04,982] Trial 98 finished with value: 0.7262129244656023 and parameters: {'learning_rate': 0.020005168816392895, 'max_depth': 32, 'random_state': 9013, 'gamma': 0.15249453496971185, 'min_child_weight': 0.1375414918769597, 'subsample': 0.04846493331040685, 'colsample_bytree': 0.460328947142462, 'colsample_bylevel': 0.38430171366648724, 'colsample_bynode': 0.6072225152426606, 'reg_alpha': 0.7794148015487848, 'reg_lambda': 0.9822654060078785, 'max_leaves': 2118}. Best is trial 24 with value: 0.7602532496638279.


rmse = 0.7262129244656023
[0]	validation_0-auc:0.746282
Will train until validation_0-auc hasn't improved in 1000 rounds.
[100]	validation_0-auc:0.763603
[200]	validation_0-auc:0.768093
[300]	validation_0-auc:0.774174
[400]	validation_0-auc:0.758467
[500]	validation_0-auc:0.758219
[600]	validation_0-auc:0.763518
[700]	validation_0-auc:0.747596
[800]	validation_0-auc:0.767923
[900]	validation_0-auc:0.765916
[999]	validation_0-auc:0.780531


[I 2021-02-23 10:45:07,405] Trial 99 finished with value: 0.7211537824607578 and parameters: {'learning_rate': 0.7133027727482708, 'max_depth': 28, 'random_state': 9584, 'gamma': 0.44219553913588017, 'min_child_weight': 0.00261084547830856, 'subsample': 0.026128080090109602, 'colsample_bytree': 0.47205759972009215, 'colsample_bylevel': 0.3975623607363843, 'colsample_bynode': 0.6245074919669346, 'reg_alpha': 0.8562032475737011, 'reg_lambda': 0.9507224793631113, 'max_leaves': 4041}. Best is trial 24 with value: 0.7602532496638279.


rmse = 0.7211537824607578
Number of finished trials: 100
Best trial:
  Value: 0.7602532496638279
  Params: 
    learning_rate: 0.00741616214412759
    max_depth: 21
    random_state: 8678
    gamma: 0.1458394749122866
    min_child_weight: 0.19427913022754295
    subsample: 0.002422155511128032
    colsample_bytree: 0.71538052580844
    colsample_bylevel: 0.8430250327565448
    colsample_bynode: 0.8344001149646294
    reg_alpha: 0.8621409873094561
    reg_lambda: 0.728185377743096
    max_leaves: 2530


In [ ]:
xgb1 = {
                'colsample_bylevel': 0.8430250327565448,
                'colsample_bynode': 0.8344001149646294,
                'colsample_bytree': 0.71538052580844,
                'gamma': 0.1458394749122866,
                'learning_rate': 0.00741616214412759,
                'max_depth': 21,
                'max_leaves': 2530,
                'min_child_weight': 0.19427913022754295,
                'random_state': 8678,
                'reg_alpha': 0.8621409873094561,
                'reg_lambda': 0.728185377743096,
                'subsample': 0.002422155511128032,
                "objective": "binary:logistic",
                #'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
         }

xgb2 = {        
                'colsample_bylevel': 0.21188746247264628,
                'colsample_bynode': 0.09055574331349836,
                'colsample_bytree': 0.9042198869852377,
                'gamma': 0.5864538038989177,
                'learning_rate': 0.719913215068412,
                'max_depth': 5,
                'max_leaves': 4225,
                'min_child_weight': 0.7932002232021746,
                'random_state': 8174,
                'reg_alpha': 0.301799671337011,
                'reg_lambda': 0.3981002753245956,
                'subsample': 0.8757297310340725,
                 'objective': "binary:logistic",
                #'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000
        }

xgb3 = {
                'colsample_bylevel': 0.8229726472603667,
                'colsample_bynode': 0.04092225582521414,
                'colsample_bytree': 0.3699851288058192,
                'gamma': 0.8895382829484678,
                'learning_rate': 0.053988146407440915,
                'max_depth': 6,
                'max_leaves': 966,
                'min_child_weight': 0.4006115309183503,
                'random_state': 4458,
                'reg_alpha': 0.7316622513065524,
                'reg_lambda': 0.9325363137123942,
                'subsample': 0.0008843533494742768,
                "objective": "binary:logistic",
                #'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
        }

In [ ]:
model_xgb1 = xgb.XGBClassifier(**xgb1)
model_xgb2 = xgb.XGBClassifier(**xgb2)
model_xgb3 = xgb.XGBClassifier(**xgb3)


model_xgb1.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)
model_xgb2.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)
model_xgb3.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)


val_x1 = model_xgb1.predict(X_test.iloc[:, 1:].values)
val_x2 = model_xgb2.predict(X_test.iloc[:, 1:].values)
val_x3 = model_xgb3.predict(X_test.iloc[:, 1:].values)

val_xgb = (val_x1 + val_x2 + val_x3) / 3

print("\n\nROC score:", metrics.roc_auc_score(y_test, val_xgb))

[0]	validation_0-auc:0.40055
Will train until validation_0-auc hasn't improved in 1000 rounds.
[500]	validation_0-auc:0.833052
[999]	validation_0-auc:0.833083
[0]	validation_0-auc:0.717854
Will train until validation_0-auc hasn't improved in 1000 rounds.
[500]	validation_0-auc:0.787928
[999]	validation_0-auc:0.78244
[0]	validation_0-auc:0.5
Will train until validation_0-auc hasn't improved in 1000 rounds.
[500]	validation_0-auc:0.798475
[999]	validation_0-auc:0.799404
ROC score: 0.7898336362010453


In [ ]:
'''
def objective(trial):    
    params = {
                'learning_rate': trial.suggest_uniform('learning_rate', 0.00001, 1.0),
                'max_depth': trial.suggest_int('max_depth', 1, 50),
                #'max_delta_step': trial.suggest_int('max_delta_step', 1, 10000),
                # trial.suggest_int('n_estimators', 1, 10000),
                'random_state': trial.suggest_int('random_state', 1, 10000),
                #'num_parallel_tree': trial.suggest_int('num_parallel_tree', 1, 1000),
                'gamma': trial.suggest_uniform('gamma', 0.00001, 1.0),
                'min_child_weight': trial.suggest_uniform('min_child_weight', 0.00001, 1.0),
                'subsample': trial.suggest_uniform('subsample', 0.00001, 1.0),
                'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.00001, 1.0),
                'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.00001, 1.0),
                'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.00001, 1.0),
                'reg_alpha': trial.suggest_uniform('reg_alpha', 0.00001, 1.0),
                'reg_lambda': trial.suggest_uniform('reg_lambda', 0.00001, 1.0),
                #'min_child_weight': trial.suggest_int('min_child_weight', 1, 10000),
                'max_leaves': trial.suggest_int('max_leaves', 1, 10000),
              
              
                "objective": "binary",
                'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
            }


    model = lgb.LGBMClassifier(**params)
    model.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=100)
    val_pred = model.predict(X_test.iloc[:, 1:].values)
    score = metrics.roc_auc_score(y_test, val_pred)
    print(f"rmse = {score}")
    return score

import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))
'''

[I 2021-02-23 11:14:09,282] A new study created in memory with name: no-name-e71daf70-e312-4533-9106-5ac008d09f31


Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.449676
[200]	valid_0's binary_logloss: 0.458018
[300]	valid_0's binary_logloss: 0.463883
[400]	valid_0's binary_logloss: 0.467074
[500]	valid_0's binary_logloss: 0.468384
[600]	valid_0's binary_logloss: 0.468958
[700]	valid_0's binary_logloss: 0.469477
[800]	valid_0's binary_logloss: 0.470075


[I 2021-02-23 11:14:09,824] Trial 0 finished with value: 0.701638963287134 and parameters: {'learning_rate': 0.5303366948507867, 'max_depth': 21, 'random_state': 4164, 'gamma': 0.10672687985697818, 'min_child_weight': 0.7485883789911895, 'subsample': 0.496454835489778, 'colsample_bytree': 0.01858712324947105, 'colsample_bylevel': 0.32064603228018657, 'colsample_bynode': 0.353627056795524, 'reg_alpha': 0.8928053285498629, 'reg_lambda': 0.011220274307156124, 'max_leaves': 4693}. Best is trial 0 with value: 0.701638963287134.


[900]	valid_0's binary_logloss: 0.470617
[1000]	valid_0's binary_logloss: 0.470885
Did not meet early stopping. Best iteration is:
[117]	valid_0's binary_logloss: 0.447735
rmse = 0.701638963287134
Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.666219
[200]	valid_0's binary_logloss: 0.681642
[300]	valid_0's binary_logloss: 0.681642
[400]	valid_0's binary_logloss: 0.681642
[500]	valid_0's binary_logloss: 0.681642
[600]	valid_0's binary_logloss: 0.681642
[700]	valid_0's binary_logloss: 0.681642
[800]	valid_0's binary_logloss: 0.681642


[I 2021-02-23 11:14:10,529] Trial 1 finished with value: 0.7244699631038678 and parameters: {'learning_rate': 0.8711695680132333, 'max_depth': 11, 'random_state': 7002, 'gamma': 0.9846968075803714, 'min_child_weight': 0.245290097400046, 'subsample': 0.466305197113972, 'colsample_bytree': 0.7895962496662519, 'colsample_bylevel': 0.223617559406103, 'colsample_bynode': 0.44569586360367247, 'reg_alpha': 0.9136964140790874, 'reg_lambda': 0.262584242083633, 'max_leaves': 9095}. Best is trial 1 with value: 0.7244699631038678.


...

[900]	valid_0's binary_logloss: 0.526632
[1000]	valid_0's binary_logloss: 0.53353
Did not meet early stopping. Best iteration is:
[66]	valid_0's binary_logloss: 0.426543
rmse = 0.7225771319053286
Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.449264
[200]	valid_0's binary_logloss: 0.479861
[300]	valid_0's binary_logloss: 0.502788
[400]	valid_0's binary_logloss: 0.521152
[500]	valid_0's binary_logloss: 0.539218
[600]	valid_0's binary_logloss: 0.555285
[700]	valid_0's binary_logloss: 0.562819


[I 2021-02-23 11:15:49,091] Trial 98 finished with value: 0.7134516320632025 and parameters: {'learning_rate': 0.13672688051026563, 'max_depth': 43, 'random_state': 6118, 'gamma': 0.38141238196482086, 'min_child_weight': 0.297591058651548, 'subsample': 0.52153948123893, 'colsample_bytree': 0.369100194843036, 'colsample_bylevel': 0.41295578849641534, 'colsample_bynode': 0.02228175534310914, 'reg_alpha': 0.7316171641935149, 'reg_lambda': 0.10647448197003853, 'max_leaves': 1401}. Best is trial 21 with value: 0.7545615079058409.


[800]	valid_0's binary_logloss: 0.563172
[900]	valid_0's binary_logloss: 0.563172
[1000]	valid_0's binary_logloss: 0.563172
Did not meet early stopping. Best iteration is:
[37]	valid_0's binary_logloss: 0.42938
rmse = 0.7134516320632025
Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.665608
[200]	valid_0's binary_logloss: 0.641676
[300]	valid_0's binary_logloss: 0.620433
[400]	valid_0's binary_logloss: 0.601858
[500]	valid_0's binary_logloss: 0.585225
[600]	valid_0's binary_logloss: 0.570461
[700]	valid_0's binary_logloss: 0.55732
[800]	valid_0's binary_logloss: 0.545405
[900]	valid_0's binary_logloss: 0.534646
[1000]	valid_0's binary_logloss: 0.524906
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.524906


[I 2021-02-23 11:15:50,988] Trial 99 finished with value: 0.7436519221979584 and parameters: {'learning_rate': 0.000822565622965601, 'max_depth': 5, 'random_state': 5159, 'gamma': 0.5012959503560478, 'min_child_weight': 0.15122791624076798, 'subsample': 0.039044213025266505, 'colsample_bytree': 0.42296442881840784, 'colsample_bylevel': 0.23608455898950542, 'colsample_bynode': 0.19622457346908212, 'reg_alpha': 0.8057136836320151, 'reg_lambda': 0.2298392842289857, 'max_leaves': 239}. Best is trial 21 with value: 0.7545615079058409.


rmse = 0.7436519221979584
Number of finished trials: 100
Best trial:
  Value: 0.7545615079058409
  Params: 
    learning_rate: 0.02644179586096865
    max_depth: 1
    random_state: 5311
    gamma: 0.5722353026832225
    min_child_weight: 0.12435531800024671
    subsample: 0.739238081306648
    colsample_bytree: 0.3947732193157685
    colsample_bylevel: 0.9913821639554284
    colsample_bynode: 0.9954352846588813
    reg_alpha: 0.622839741687606
    reg_lambda: 0.08427712683995517
    max_leaves: 122


In [ ]:
lgb1 = {
                'colsample_bylevel': 0.17415390952282833,
                'colsample_bynode': 0.23146367015162006,
                'colsample_bytree': 0.7600539666086275,
                'gamma': 0.422250673515022,
                'learning_rate': 0.027735233822329265,
                'max_depth': 1,
                'max_leaves': 6381,
                'min_child_weight': 0.6629013655404066,
                'random_state': 1072,
                'reg_alpha': 0.7178319913875806,
                'reg_lambda': 0.5425881137145795,
                'subsample': 0.6589712226735368,
                "objective": "binary",
                #'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
         }

lgb2 = {
                'colsample_bylevel': 0.9987514746936683,
                'colsample_bynode': 0.5368441576747792,
                'colsample_bytree': 0.5578754500207318,
                'gamma': 0.5274145503795864,
                'learning_rate': 0.044050940942348515,
                'max_depth': 1,
                'max_leaves': 4834,
                'min_child_weight': 0.38912178269827274,
                'random_state': 7338,
                'reg_alpha': 0.9578154029845921,
                'reg_lambda': 0.24406921353489225,
                'subsample': 0.10328388796537287,
                "objective": "binary",
                #'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
         }

lgb3 = {
                'colsample_bylevel': 0.8620482807826502,
                'colsample_bynode': 0.6336815413907267,
                'colsample_bytree': 0.35120639119912106,
                'gamma': 0.24403741528670408,
                'learning_rate': 0.025885558841873723,
                'max_depth': 1,
                'max_leaves': 385,
                'min_child_weight': 0.19951436583130955,
                'random_state': 5180,
                'reg_alpha': 0.5979999102002361,
                'reg_lambda': 0.16672922184197916,
                'subsample': 0.7024449173883872,
                'objective': "binary",
                #'tree_method': 'gpu_hist',
                'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'auc',
                'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
         }

In [ ]:
model_lgb1 = lgb.LGBMClassifier(**lgb1)
model_lgb2 = lgb.LGBMClassifier(**lgb2)
model_lgb3 = lgb.LGBMClassifier(**lgb3)


model_lgb1.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)
model_lgb2.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)
model_lgb3.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)


val_l1 = model_lgb1.predict(X_test.iloc[:, 1:].values)
val_l2 = model_lgb2.predict(X_test.iloc[:, 1:].values)
val_l3 = model_lgb3.predict(X_test.iloc[:, 1:].values)

val_lgb = (val_l1 + val_l2 + val_l3) / 3

print("\n\nROC score:", metrics.roc_auc_score(y_test, val_lgb))

Training until validation scores don't improve for 1000 rounds.
[500]	valid_0's binary_logloss: 0.46258
[1000]	valid_0's binary_logloss: 0.44444
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.44444
Training until validation scores don't improve for 1000 rounds.
[500]	valid_0's binary_logloss: 0.448957
[1000]	valid_0's binary_logloss: 0.434951
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.434951
Training until validation scores don't improve for 1000 rounds.
[500]	valid_0's binary_logloss: 0.464239
[1000]	valid_0's binary_logloss: 0.445245
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.445245


ROC score: 0.7534431421991508


In [ ]:
def objective(trial):    
    params = {
                'learning_rate': trial.suggest_uniform('learning_rate', 0.00001, 1.0),
                'max_depth': trial.suggest_int('max_depth', 1, 16),
                #'max_delta_step': trial.suggest_int('max_delta_step', 1, 10000),
                # trial.suggest_int('n_estimators', 1, 10000),
                'random_state': trial.suggest_int('random_state', 1, 10000),
                #'num_parallel_tree': trial.suggest_int('num_parallel_tree', 1, 1000),
                #'gamma': trial.suggest_uniform('gamma', 0.00001, 1.0),
                #'min_child_weight': trial.suggest_uniform('min_child_weight', 0.00001, 1.0),
                'subsample': trial.suggest_uniform('subsample', 0.00001, 1.0),
                #'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.00001, 1.0),
                'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.00001, 1.0),
                #'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.00001, 1.0),
                #'reg_alpha': trial.suggest_uniform('reg_alpha', 0.00001, 1.0),
                #'reg_lambda': trial.suggest_uniform('reg_lambda', 0.00001, 1.0),
                #'min_child_weight': trial.suggest_int('min_child_weight', 1, 10000),
                #'max_leaves': trial.suggest_int('max_leaves', 1, 10000),
              
              
                "objective": "Logloss",
                #'tree_method': 'gpu_hist',
                #'booster': 'gbtree', #, gblinear or dart.
                'eval_metric': 'AUC',
                #'importance_type': 'gain',  #“gain”, “weight”, “cover”, “total_gain” or “total_cover”
                'n_estimators': 1000,
            }


    model = CatBoostClassifier(**params)
    model.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=100)
    val_pred = model.predict(X_test.iloc[:, 1:].values)
    score = metrics.roc_auc_score(y_test, val_pred)
    print(f"rmse = {score}")
    return score

import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2021-02-23 16:09:50,424] A new study created in memory with name: no-name-9a7bec61-76a2-4bdf-b40f-1d236130b3c6


0:	test: 0.7802042	best: 0.7802042 (0)	total: 28.2ms	remaining: 28.2s
100:	test: 0.7751469	best: 0.7991523 (2)	total: 1.72s	remaining: 15.3s
200:	test: 0.7749612	best: 0.7991523 (2)	total: 3.37s	remaining: 13.4s
300:	test: 0.7758527	best: 0.7991523 (2)	total: 5.32s	remaining: 12.3s
400:	test: 0.7764522	best: 0.7991523 (2)	total: 6.99s	remaining: 10.4s
500:	test: 0.7764709	best: 0.7991523 (2)	total: 8.47s	remaining: 8.43s
600:	test: 0.7761557	best: 0.7991523 (2)	total: 9.91s	remaining: 6.58s
700:	test: 0.7761988	best: 0.7991523 (2)	total: 11.4s	remaining: 4.84s
800:	test: 0.7765493	best: 0.7991523 (2)	total: 12.8s	remaining: 3.17s
900:	test: 0.7765234	best: 0.7991523 (2)	total: 14.2s	remaining: 1.56s


[I 2021-02-23 16:10:06,384] Trial 0 finished with value: 0.7113537303513192 and parameters: {'learning_rate': 0.8634338640798637, 'max_depth': 9, 'random_state': 5480, 'subsample': 0.5687823167007227, 'colsample_bylevel': 0.34086935359063114}. Best is trial 0 with value: 0.7113537303513192.


999:	test: 0.7764582	best: 0.7991523 (2)	total: 15.7s	remaining: 0us

bestTest = 0.7991523384
bestIteration = 2

Shrink model to first 3 iterations.
rmse = 0.7113537303513192
0:	test: 0.7909644	best: 0.7909644 (0)	total: 6.88ms	remaining: 6.88s
100:	test: 0.8214791	best: 0.8373832 (15)	total: 564ms	remaining: 5.02s
200:	test: 0.8069182	best: 0.8373832 (15)	total: 1.1s	remaining: 4.39s
300:	test: 0.8009439	best: 0.8373832 (15)	total: 1.66s	remaining: 3.86s
400:	test: 0.7976934	best: 0.8373832 (15)	total: 2.21s	remaining: 3.31s
500:	test: 0.7953222	best: 0.8373832 (15)	total: 2.85s	remaining: 2.84s
600:	test: 0.7944382	best: 0.8373832 (15)	total: 3.44s	remaining: 2.28s
700:	test: 0.7903743	best: 0.8373832 (15)	total: 3.98s	remaining: 1.7s
800:	test: 0.7899653	best: 0.8373832 (15)	total: 4.51s	remaining: 1.12s
900:	test: 0.7884522	best: 0.8373832 (15)	total: 5.04s	remaining: 554ms


[I 2021-02-23 16:10:12,171] Trial 1 finished with value: 0.7372334635333291 and parameters: {'learning_rate': 0.3810573726245071, 'max_depth': 5, 'random_state': 8446, 'subsample': 0.24584897127296423, 'colsample_bylevel': 0.7573052229117017}. Best is trial 1 with value: 0.7372334635333291.


...

999:	test: 0.8219560	best: 0.8395427 (213)	total: 2.43s	remaining: 0us

bestTest = 0.8395426734
bestIteration = 213

Shrink model to first 214 iterations.
rmse = 0.7243112611588166
0:	test: 0.7642605	best: 0.7642605 (0)	total: 2.65ms	remaining: 2.65s
100:	test: 0.8373208	best: 0.8376051 (97)	total: 270ms	remaining: 2.41s
200:	test: 0.8372830	best: 0.8380796 (129)	total: 539ms	remaining: 2.14s
300:	test: 0.8351774	best: 0.8380796 (129)	total: 820ms	remaining: 1.9s
400:	test: 0.8313937	best: 0.8380796 (129)	total: 1.1s	remaining: 1.65s
500:	test: 0.8283350	best: 0.8380796 (129)	total: 1.38s	remaining: 1.37s
600:	test: 0.8271278	best: 0.8380796 (129)	total: 1.64s	remaining: 1.09s
700:	test: 0.8249507	best: 0.8380796 (129)	total: 1.92s	remaining: 821ms
800:	test: 0.8240296	best: 0.8380796 (129)	total: 2.18s	remaining: 542ms
900:	test: 0.8227211	best: 0.8380796 (129)	total: 2.44s	remaining: 268ms


[I 2021-02-23 17:34:12,262] Trial 98 finished with value: 0.7289023014266063 and parameters: {'learning_rate': 0.2047773462692397, 'max_depth': 3, 'random_state': 6042, 'subsample': 0.49944451642864074, 'colsample_bylevel': 0.09153706872572126}. Best is trial 41 with value: 0.7511931626234012.


999:	test: 0.8211700	best: 0.8380796 (129)	total: 2.7s	remaining: 0us

bestTest = 0.8380795795
bestIteration = 129

Shrink model to first 130 iterations.
rmse = 0.7289023014266063
0:	test: 0.6853283	best: 0.6853283 (0)	total: 2.33ms	remaining: 2.33s
100:	test: 0.8361418	best: 0.8373970 (81)	total: 263ms	remaining: 2.34s
200:	test: 0.8360325	best: 0.8375193 (153)	total: 511ms	remaining: 2.03s
300:	test: 0.8331579	best: 0.8375193 (153)	total: 756ms	remaining: 1.76s
400:	test: 0.8318552	best: 0.8375193 (153)	total: 1.01s	remaining: 1.51s
500:	test: 0.8300225	best: 0.8375193 (153)	total: 1.27s	remaining: 1.27s
600:	test: 0.8279787	best: 0.8375193 (153)	total: 1.52s	remaining: 1.01s
700:	test: 0.8269086	best: 0.8375193 (153)	total: 1.77s	remaining: 756ms
800:	test: 0.8257908	best: 0.8375193 (153)	total: 2.03s	remaining: 504ms
900:	test: 0.8243175	best: 0.8375193 (153)	total: 2.29s	remaining: 252ms


[I 2021-02-23 17:34:14,980] Trial 99 finished with value: 0.7288978853724831 and parameters: {'learning_rate': 0.31998840777041765, 'max_depth': 2, 'random_state': 4715, 'subsample': 0.4526930598464044, 'colsample_bylevel': 0.1122986684165952}. Best is trial 41 with value: 0.7511931626234012.


999:	test: 0.8229093	best: 0.8375193 (153)	total: 2.54s	remaining: 0us

bestTest = 0.8375192926
bestIteration = 153

Shrink model to first 154 iterations.
rmse = 0.7288978853724831
Number of finished trials: 100
Best trial:
  Value: 0.7511931626234012
  Params: 
    learning_rate: 0.06782171217597968
    max_depth: 2
    random_state: 5033
    subsample: 0.9384009973690735
    colsample_bylevel: 0.009301667829591798


In [ ]:
cat1 = {
                'colsample_bylevel': 0.9987514746936683,
                'learning_rate': 0.044050940942348515,
                'max_depth': 1,
                'random_state': 4834,
                'subsample': 0.10328388796537287,
                'objective': "Logloss",
                'eval_metric': 'AUC',
                'n_estimators': 1000,
         }

cat2 = {
                'colsample_bylevel': 0.17415390952282833,
                'learning_rate': 0.027735233822329265,
                'max_depth': 1,
                'random_state': 1072,
                'subsample': 0.6589712226735368,
                'objective': "Logloss",
                'eval_metric': 'AUC',
                'n_estimators': 1000,
         }

cat3 = {
                'colsample_bylevel': 0.8620482807826502,
                'learning_rate': 0.025885558841873723,
                'max_depth': 1,
                'random_state': 5180,
                'subsample': 0.7024449173883872,
                'objective': "Logloss",
                'eval_metric': 'AUC',
                'n_estimators': 1000,
         }

In [ ]:
model_cat1 = CatBoostClassifier(**cat1)
model_cat2 = CatBoostClassifier(**cat2)
model_cat3 = CatBoostClassifier(**cat3)


model_cat1.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)
model_cat2.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)
model_cat3.fit(X, y, eval_set=[(X_test.iloc[:, 1:].values, y_test)], early_stopping_rounds=1000, verbose=500)


val_c1 = model_cat1.predict_proba(X_test.iloc[:, 1:].values)
val_c2 = model_cat2.predict_proba(X_test.iloc[:, 1:].values)
val_c3 = model_cat3.predict_proba(X_test.iloc[:, 1:].values)

val_cat = (val_c1[:, -1] + val_c2[:, -1] + val_c3[:, -1]) / 3

print("\n\nROC score:", metrics.roc_auc_score(y_test, val_cat))

0:	test: 0.7223227	best: 0.7223227 (0)	total: 2.84ms	remaining: 2.84s
500:	test: 0.8336232	best: 0.8336481 (497)	total: 1.29s	remaining: 1.28s
999:	test: 0.8372430	best: 0.8372458 (990)	total: 2.55s	remaining: 0us

bestTest = 0.8372457733
bestIteration = 990

Shrink model to first 991 iterations.
0:	test: 0.6841042	best: 0.6841042 (0)	total: 2.51ms	remaining: 2.51s
500:	test: 0.8300752	best: 0.8301602 (492)	total: 1.19s	remaining: 1.19s
999:	test: 0.8344035	best: 0.8344035 (999)	total: 2.38s	remaining: 0us

bestTest = 0.8344034904
bestIteration = 999

0:	test: 0.7223227	best: 0.7223227 (0)	total: 2.75ms	remaining: 2.75s
500:	test: 0.8300493	best: 0.8300493 (500)	total: 1.34s	remaining: 1.34s
999:	test: 0.8348015	best: 0.8348517 (975)	total: 2.67s	remaining: 0us

bestTest = 0.8348517199
bestIteration = 975

Shrink model to first 976 iterations.


ROC score: 0.8357241666353863


In [ ]:
ensem = (val_lgb + val_xgb + val_cat) / 3
print("\n\nROC score:", metrics.roc_auc_score(y_test, ensem))



ROC score: 0.8326188525766572


In [ ]:
'''

hidden_layers = 1 
neuron_numbers = 30 
rate_of_learning = 3e-3 


def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})


def ModelTuner1(drop_rate, hiddenN = hidden_layers, neuronsN = neuron_numbers, learnR = rate_of_learning):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[19]))
    model.add(keras.layers.BatchNormalization())
    for layers in range(hiddenN):
        model.add(keras.layers.Dense(neuronsN, activation="gelu"))
        model.add(keras.layers.Dropout(drop_rate))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation="sigmoid"))

    optim = keras.optimizers.Nadam(lr = learnR)
    model.compile(loss="binary_crossentropy", optimizer = optim, metrics = ["AUC"])
    return model

params = {
            "hiddenN": [1, 2, 3],
            "neuronsN": np.arange(1, 1000),
            "learnR": reciprocal(3e-4, 3e-2),
            "drop_rate": reciprocal(1e-1, 5e-1)
          }

reg_keras = keras.wrappers.scikit_learn.KerasRegressor(ModelTuner1)
rsCV = RandomizedSearchCV(reg_keras, params, n_iter=100, cv=3)
print(rsCV.best_params_,"\n\n\n", rsCV.best_score_)
model = rsCV.best_estimator_.model
'''

In [ ]:
rsCV.fit(X, y, epochs=10, validation_data = (X_test.iloc[:, 1:], y_test), callbacks=[keras.callbacks.EarlyStopping(patience = 5)])

Streaming output truncated to the last 5000 lines.
Epoch 5/10
123/123 [==============================] - 1s 5ms/step - loss: 0.4172 - auc: 0.8380 - val_loss: 0.4485 - val_auc: 0.8323
Epoch 6/10
123/123 [==============================] - 1s 5ms/step - loss: 0.4235 - auc: 0.8412 - val_loss: 0.4508 - val_auc: 0.8324
Epoch 7/10
123/123 [==============================] - 1s 6ms/step - loss: 0.4228 - auc: 0.8358 - val_loss: 0.4492 - val_auc: 0.8274
Epoch 8/10
123/123 [==============================] - 1s 5ms/step - loss: 0.4204 - auc: 0.8315 - val_loss: 0.4620 - val_auc: 0.8272
Epoch 9/10
123/123 [==============================] - 1s 6ms/step - loss: 0.4172 - auc: 0.8454 - val_loss: 0.4463 - val_auc: 0.8354
Epoch 10/10
62/62 [==============================] - 1s 3ms/step - loss: 0.6718 - auc: 0.8436
Epoch 1/10
123/123 [==============================] - 8s 40ms/step - loss: 0.5695 - auc: 0.8131 - val_loss: 1.9643 - val_auc: 0.7708
Epoch 2/10
123/123 [==============================] - 4s 36ms/

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f1e06dbcc50>,
                   iid='warn', n_iter=100, n_jobs=None,
                   param_distributions={'drop_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f1e06dbcfd0>,
                                        'hiddenN': [1, 2, 3],
                                        'learnR': <scipy.stats._distn_infrastructure.rv...
       937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949,
       950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962,
       963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975,
       976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988,
       989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, ver

In [ ]:
'''{'drop_rate': 0.41955910741026337, 'hiddenN': 3, 'learnR': 0.013784026384995006, 'neuronsN': 880} '''

def ModelTuned(drop_rate = 0.41955910741026337, hiddenN = 3, neuronsN = 880, learnR = 0.013784026384995006):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[19]))
    model.add(keras.layers.BatchNormalization())
    for layers in range(hiddenN):
        model.add(keras.layers.Dense(neuronsN, activation="gelu"))
        model.add(keras.layers.Dropout(drop_rate))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation="sigmoid"))

    optim = keras.optimizers.Nadam(lr = learnR)
    model.compile(loss="binary_crossentropy", optimizer = optim, metrics = ["AUC"])
    return model

model = ModelTuned()
model.fit(X, y, epochs=30, validation_data = (X_test.iloc[:, 1:], y_test), callbacks=[keras.callbacks.EarlyStopping(patience = 5)], batch_size = 4096)

Epoch 1/30
2/2 [==============================] - 5s 1s/step - loss: 1.7285 - auc: 0.4704 - val_loss: 31.1166 - val_auc: 0.5880
Epoch 2/30
2/2 [==============================] - 2s 750ms/step - loss: 1.9398 - auc: 0.7718 - val_loss: 2.6250 - val_auc: 0.7712
Epoch 3/30
2/2 [==============================] - 2s 753ms/step - loss: 0.6487 - auc: 0.8213 - val_loss: 1.4165 - val_auc: 0.8017
Epoch 4/30
2/2 [==============================] - 2s 735ms/step - loss: 0.4977 - auc: 0.8338 - val_loss: 1.9819 - val_auc: 0.8026
Epoch 5/30
2/2 [==============================] - 2s 749ms/step - loss: 0.5409 - auc: 0.8388 - val_loss: 2.5660 - val_auc: 0.8000
Epoch 6/30
2/2 [==============================] - 2s 745ms/step - loss: 0.6049 - auc: 0.8513 - val_loss: 0.8635 - val_auc: 0.8221
Epoch 7/30
2/2 [==============================] - 2s 746ms/step - loss: 0.4593 - auc: 0.8608 - val_loss: 0.9916 - val_auc: 0.8221
Epoch 8/30
2/2 [==============================] - 2s 745ms/step - loss: 0.4604 - auc: 0.8614

In [ ]:
preds = model.predict(X_test.iloc[:, 1:])

from sklearn.metrics import accuracy_score, roc_auc_score
print(accuracy_score(y_test, np.round(preds)))
print(roc_auc_score(y_test, preds))

0.6184686164968781
0.831732329711433


In [ ]:
final = (val_cat + preds[:, -1]) / 2
print(accuracy_score(y_test, np.round(final)))
print(roc_auc_score(y_test, final))

0.7029247453171212
0.8404532085945245


In [ ]:
{'drop_rate': 0.1406357539732016, 'hiddenN': 1, 'learnR': 0.009030930388880937, 'neuronsN': 28} 